# 共通処理

In [ ]:
import os
import sys

import h5py

"""
このnotebookがbaseフォルダを読めるようになるための処理
"""

root_rel = '../../'

# 相対パスを絶対パスに変換してsys.pathに追加
root_abs = os.path.abspath(root_rel)
if root_abs not in sys.path:
    sys.path.append(root_abs)
print(root_abs)

In [ ]:
import json

try:
    with open('setting.json', 'r') as file:
        setting = json.load(file)
except FileNotFoundError:
    print("先にset_run.ipynbで解析runを設定してください")

hdf5_path = setting['hdf5_path']
hdf5_path

run_name = hdf5_path.split('/')[-1].split('.')[0]

hdf5_path, run_name

In [ ]:
from model.File.HDF5 import HDF5

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

In [ ]:
plt.rcParams['font.family'] = 'Arial'
plt.rcParams['mathtext.fontset'] = 'cm'     #数式用のフォントejavuserif" or "cm"
plt.rcParams['xtick.direction'] = 'in'      #x軸の目盛線 #内in')か外向き('out')か双方向か('inout')
plt.rcParams['ytick.direction'] = 'in'      #y軸の目盛線 #内in')か外向き('out')か双方向か('inout')
plt.rcParams['xtick.major.width'] = 1.0     #x軸主目盛り線の線幅
plt.rcParams['ytick.major.width'] = 1.0     #y軸主目盛り線の線幅
plt.rcParams['font.size'] = 18               #フォントの大きさ
plt.rcParams['axes.linewidth'] = 0.8        #軸の線幅edge linewidth。囲みの太さ
plt.rcParams['figure.dpi'] = 300
plt.rcParams['figure.figsize'] = (8, 6)


In [ ]:
# hdf5内のlayer pathを取得する
with h5py.File(hdf5_path, 'r') as f_read:
    hdf = HDF5(f=f_read)

In [ ]:
def return_hdf5_data(hdf5_path: str, *, query: str, shape=None):
    with h5py.File(hdf5_path, 'r') as f_read:
        hdf = HDF5(f=f_read)
        
        to_data = hdf.search_data_path(query=query)
        
        if type(to_data) is str:
            return hdf.return_data(data_path=to_data, shape=shape)
        elif type(to_data) is list:
            raise Exception("複数のlayer pathが見つかりました。一括で返して欲しい場合は実装してください。")
        else:
            raise Exception("layer pathが見つかりませんでした。")

# 格子体積を計算する

In [ ]:
# TODO 汚すぎるのであとで整形する

## データの取得

In [ ]:
# 軽いデータ
fps = return_hdf5_data(hdf5_path, query='nxs/fps')
exposure_ms = return_hdf5_data(hdf5_path, query='nxs/exposure_ms')
frame_num = return_hdf5_data(hdf5_path, query='nxs/frame_num')

tth_arr = return_hdf5_data(hdf5_path, query='nxs/tth')
azi_arr = return_hdf5_data(hdf5_path, query='nxs/azi')

time_arr = np.arange(frame_num) / fps

In [ ]:
# 強度
I = return_hdf5_data(hdf5_path, query='nxs/I')

In [ ]:
# 温度も取得しておく
T_up = return_hdf5_data(hdf5_path, query='T/T_up_mean')
T_down = return_hdf5_data(hdf5_path, query='T/T_down_mean')

T_up_err_minus = return_hdf5_data(hdf5_path, query='T/T_up_err_minus')
T_up_err_plus = return_hdf5_data(hdf5_path, query='T/T_up_err_plus')
T_down_err_minus = return_hdf5_data(hdf5_path, query='T/T_down_err_minus')
T_down_err_plus = return_hdf5_data(hdf5_path, query='T/T_down_err_plus')

## 強度のplot

In [ ]:
df_I = pd.DataFrame(I)

plt.plot(tth_arr, df_I.sum())
plt.grid()
plt.show()
plt.close()

In [ ]:
I.shape

In [ ]:
# 1d patternの時間進化を確認する
plt.imshow(I, aspect='auto', cmap='jet', origin='lower',
           extent=[tth_arr[0], tth_arr[-1], time_arr[0], time_arr[-1]])
plt.colorbar()
plt.show()
plt.close()

## 角度を狭めて見ていく

In [ ]:
# tthを指定したら、そのindexを返す関数
def return_idx(arr, value):
    idx = (np.abs(arr - value)).argmin()
    return idx

In [ ]:
return_idx(tth_arr, 10)

In [ ]:
# 特定のピークとか見る
from_tth = 9.7
to_tth = 10.7

from_idx = return_idx(tth_arr, from_tth)
to_idx = return_idx(tth_arr, to_tth)
plt.imshow(I[:, from_idx:to_idx].T, aspect='auto', cmap='jet', origin='lower',
           extent=[time_arr[0], time_arr[-1], from_tth, to_tth])
plt.colorbar()
plt.show()
plt.close()


In [ ]:
fig, ax = plt.subplots()
ax.plot(time_arr, df_I.iloc[:, from_idx:to_idx].max(axis=1))
twin = ax.twinx()
twin.plot(time_arr, T_up)
twin.plot(time_arr, T_down)
twin.set_ylim(1_000, 6_000)
plt.show()
plt.close()

## 回折角を得る

In [ ]:
# 最大値をえる
idxmax_arr = df_I.iloc[:, from_idx:to_idx].idxmax(axis=1)
print(idxmax_arr.shape)

maxtth_list = []
for frame in range(frame_num):
    if I[frame, from_idx:to_idx].max() == 0:
        maxtth = maxtth_list[-1]
    else:
        maxtth = tth_arr[(idxmax_arr[frame])]
    maxtth_list.append(maxtth)
kcl_tth_arr = np.array(maxtth_list)

# center_list = []
# indices = np.arange(from_idx, to_idx)
# for frame in tqdm(range(frame_num)):
#     intensities = I[frame, from_idx:to_idx]
#     if np.sum(intensities) == 0:
#         maxtth = tth_arr[(idxmax_arr[frame])]
#         center_list.append(maxtth) 
#     else:
#         center_idx = int(np.sum(indices * intensities) / np.sum(intensities))
#         center_list.append(tth_arr[center_idx])
# center_arr = np.array([center_list])[0]

plt.plot(time_arr, kcl_tth_arr)
# plt.plot(time_arr, center_arr)
plt.show()
plt.close()

In [ ]:
# 格子体積を計算

nxs_path = return_hdf5_data(hdf5_path, query='path/nxs_path')
poni_path = return_hdf5_data(hdf5_path, query='path/poni_path')
# nxsファイルとponiファイルを読み込む
from model.XRD.Nxs import NxsFile

nxs = NxsFile(nxs_path=nxs_path, poni_path=poni_path)


In [ ]:
xray_wl = nxs.ai.wavelength
xray_wl *= 10**10
xray_wl # Å

In [ ]:
def calc_cubic_a(peak_twotheta_deg,
                 h=1,k=1,l=0,
                 wl=xray_wl):
    peak_twotheta_rad = (peak_twotheta_deg/ 360) *2 *  np.pi
    a = np.sqrt(h**2 + k**2 + l**2) * wl/(2 * np.sin(peak_twotheta_rad/2) )
    return a

In [ ]:
a = calc_cubic_a(kcl_tth_arr)

In [ ]:
plt.plot(time_arr, a)
plt.show()
plt.close()

In [ ]:
V_arr = a**3
plt.plot(time_arr, V_arr)
plt.show()
plt.close()


# 圧力を計算する

In [ ]:
from model.Calculation import eqn_vinet

In [ ]:
# KCl B2
v0 = 54.5
k0 = 17.2
k0p = 5.89

P_non_T = eqn_vinet.vinet_p(V_arr, v0, k0, k0p)

In [ ]:
fig, ax = plt.subplots()
ax.plot(time_arr, P_non_T)
twin = ax.twinx()
twin.plot(time_arr, T_up)
twin.plot(time_arr, T_down)
twin.set_ylim(1_000, 6_000)
plt.show()
plt.close()


In [ ]:
# 温度誤差を考える
# 上限と下限
T_range = [
    T_up - T_up_err_minus,
    T_up + T_up_err_plus,
    T_down - T_down_err_minus,
    T_down + T_down_err_plus,
]
df_T_range = pd.DataFrame(T_range)
df_T_range[df_T_range < 1_000] = 0
T_high_arr = df_T_range.max()
T_low_arr = df_T_range.min()

# 真ん中
T_mean = [T_up, T_down]
T_mean = pd.DataFrame(T_mean)
T_mean_arr = df_T_range.mean()


plt.plot(time_arr, T_high_arr)
plt.plot(time_arr, T_low_arr)
plt.plot(time_arr, T_mean_arr, color='black')
plt.show()
plt.close()

In [ ]:
# 温度を取り入れる
akt = 0.00224

P_max_arr = P_non_T + T_high_arr * akt
P_mean_arr = P_non_T + T_mean_arr * akt
P_min_arr = P_non_T + T_low_arr * akt

P_err = [
    P_mean_arr - P_min_arr,
    P_max_arr - P_mean_arr,
]

T_err = [
    T_mean_arr - T_low_arr,
    T_high_arr - T_mean_arr,
]

In [ ]:
plt.errorbar(x=P_mean_arr, y=T_mean_arr, yerr=T_err, xerr=P_err,
             fmt='o', c='black',mfc='white', capsize=3, elinewidth=1, alpha=0.5)
plt.xlim(0, 60)
plt.ylim(1_000, 7_000)
plt.show()
plt.close()

In [ ]:
# satuあり、全体、レーザーあり
fig, ax = plt.subplots()

ax.set_title(f"T evolution of {run_name}")

ax.errorbar(x=time_arr, y=T_mean_arr, yerr=T_err,
            c='black', marker='.',mfc='white', capsize=3, elinewidth=1, alpha=0.5,)

ax.set_ylim(1_000, 7_000)
ax.set_xlabel('Time (sec)')
ax.set_ylabel('Temperature (K)')

plt.show()
plt.close()
# for i in range(region_num):
#     idx = i * 2
#     ax.axvspan(xmin=saturation_time_arr[idx], xmax=saturation_time_arr[idx+1],
#                facecolor='red', alpha=0.3)

# twin = ax.twinx()
# twin.plot(laser_time_arr, laser_power_arr, color='black', alpha=0.3, label='Laser')
# twin.legend(loc="upper left", bbox_to_anchor=(0, 0.8))


# 保存しておく

In [ ]:
PT_layer = 'entry/PT/'

# P_mean_arr
# P_err
# T_mean_arr
# T_err


with h5py.File(hdf5_path, 'a') as f_append:
    layer = PT_layer + 'P_mean_arr'
    f_append.create_dataset(name=layer, data=P_mean_arr)
    layer = PT_layer + 'T_mean_arr'
    f_append.create_dataset(name=layer, data=T_mean_arr)

    layer = PT_layer + 'P_err'
    f_append.create_dataset(name=layer, data=P_err)
    layer = PT_layer + 'T_err'
    f_append.create_dataset(name=layer, data=T_err)
    
print("書き込み終了")

# 確認
with h5py.File(hdf5_path, 'r') as f_read:
    HDF5(f=f_read).show_all_hierarchy(f=f_read, display_length=80)
